In [8]:
cd '/home/user/project/python/rmsKit/'

/home/user/project/python/rmsKit


In [9]:

from importlib import reload
import sys
sys.path.append("/home/user/project/python/rmsKit")
import rms

import jax 
import numpy as np
import jax.numpy as jnp
from jax.config import config
sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import numpy as np

import os
groundstate = np.load("../exact/test/out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"


In [10]:
import rms
reload(rms)

<module 'rms' from '/home/user/project/python/rmsKit/rms/__init__.py'>

In [11]:
ur = rms.unitary.UnitaryRiemanGenerator(8, jax.random.PRNGKey(0), np.float64)
u = ur.reset_matrix()

UnitaryRiemanGenerator is initialized


In [ ]:
import sys
sys.path.append("/home/user/project/python/rmsKit")
from jax_lattice import KH

p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)

In [ ]:
import numpy as np
u = np.load("array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
# U = np.kron(u,u)
# U = np.kron(U,U)

In [ ]:
state = rms.loss.init_loss(jnp.array(H), 8, np.float64, "sel", beta=1.0)
state_list = [state]
qesLoss = rms.loss.system_el_multi

In [ ]:
def loss(u):
    return qesLoss(state_list, jnp.array(u))

In [ ]:
%timeit loss(jnp.array(u)).block_until_ready()

In [ ]:
%%timeit 
v, g = jax.value_and_grad(loss)(jnp.array(u))
v.block_until_ready(), g.block_until_ready()

In [10]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/user/project/python/rmsKit")
import os
import torch
from lattice import KH
# # os.environ["CUDA_VISIBLE_DEVICES"] = ""
# # os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import rms_torch
import numpy as np
# u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
u = np.load("/home/user/project/python/rmsKit/array/torch/KH/3site/sqel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1_e_50_lr_0.5/u/0.npy")
device = torch.device("cuda")
p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)
# Create an instance of the CustomModel class
E, V = np.linalg.eigh(H)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
H_gpu = torch.tensor(H, dtype=torch.float64, device=device)
model = rms_torch.UnitaryRieman(H.shape[0], 3, device=device, u0=u).to(device)
loss = rms_torch.SystemQuasiEnergyLoss(H, N = 10, device=device).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)
compiled_loss = torch.compile(loss, dynamic = False, fullgraph=True)


In [13]:
compiled_loss(compiled_model(), 100)

tensor(8.0341, device='cuda:0', dtype=torch.float64,
       grad_fn=<CompiledFunctionBackward>)

In [5]:
%%time
x = compiled_loss(compiled_model())
print(x)

tensor(7.6941, device='cuda:0', dtype=torch.float64,
       grad_fn=<CompiledFunctionBackward>)
CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 1.1 s


In [ ]:
test =  ()

In [35]:
%%time
compiled_model.reset_params()
print(next(compiled_model.parameters()).data)

tensor([[-0.3589,  0.5424, -0.2497, -0.2731,  0.0876, -0.3453,  0.4022, -0.3891],
        [-0.0693, -0.3173, -0.5213,  0.3710,  0.3618,  0.3694,  0.0897, -0.4578],
        [ 0.0471,  0.1124, -0.7220, -0.1243, -0.0719, -0.2347, -0.5561,  0.2811],
        [ 0.2508, -0.0900,  0.2653, -0.3463,  0.6088, -0.2487, -0.4199, -0.3603],
        [-0.7052, -0.5560,  0.0149, -0.3474,  0.1196, -0.0798,  0.0194,  0.2274],
        [ 0.2965, -0.4520, -0.1049,  0.2258, -0.1235, -0.7265,  0.3163, -0.0509],
        [-0.0373, -0.2100,  0.0238, -0.2446, -0.6752,  0.0755, -0.2696, -0.5998],
        [-0.4634,  0.1638,  0.2499,  0.6487, -0.0064, -0.2958, -0.4142, -0.1271]],
       device='cuda:0', dtype=torch.float64)
CPU times: user 7.32 ms, sys: 0 ns, total: 7.32 ms
Wall time: 6.92 ms


In [33]:
%%time
compiled_model.reset_params()
x = compiled_loss(compiled_model())
x.backward()
print(next(compiled_model.parameters()).grad.data)

tensor([[ -9.4834,  -5.2681,   8.1433,  22.2067,   9.3905, -19.0828,  -9.3958,
          -5.1588],
        [ -6.7860,  -0.5571, -18.1771,  -5.1018,   0.6570,   2.1333,  -1.1790,
          -6.4202],
        [  1.3660,  13.8845, -14.1871,   6.3950,   2.9877, -22.7653,  20.5231,
          -0.9765],
        [ -3.5940,  -8.2515,   7.4590, -14.7142,   1.8519, -10.9381,  26.3280,
           7.7427],
        [ -2.8593,   8.8188,  12.3703, -15.3801, -18.6747,  -1.7176,  14.5213,
          -4.7598],
        [  7.5711, -24.3100,  16.2202,   4.3194,  -0.5520, -10.4871,   6.9503,
          -6.2057],
        [  2.7612,   2.9957,  13.2747,  -3.3060, -10.9493, -27.6429,   7.6486,
           7.6357],
        [ -5.9324,  -0.5958,  15.8760,  14.6059, -21.4196,   8.0603, -19.1400,
          -4.7803]], device='cuda:0', dtype=torch.float64)
CPU times: user 2.77 s, sys: 9.76 ms, total: 2.78 s
Wall time: 2.78 s


In [8]:
U = compiled_model()

In [9]:
%timeit compiled_loss(U)

331 µs ± 107 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
# u = torch.tensor(u)
# U = torch.kron(u, u)
# U = torch.kron(U, U).to(device)
# X = torch.from_numpy(V[:,0]).to(device)
# SH = loss.stoquastic(loss.H)
# SUH = loss.stoquastic(U @ loss.H @ U.T)
# SUx = torch.abs(U @ X)
# y = SUH @ SUx
# y2 = y
# for _ in range(10):
#     y2 = SUH @ y2 
#     y2 /= torch.norm(y2)
# quasi_Sgs = torch.abs(y2)
# gap = (SUx - quasi_Sgs) @ SUH @ (quasi_Sgs + SUx) 
# gap - SUx @ y

In [3]:
compiled_loss(compiled_model())

tensor(14.0194, device='cuda:0', dtype=torch.float64,
       grad_fn=<CompiledFunctionBackward>)

In [3]:
%timeit loss(compiled_model())

3.57 ms ± 765 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit compiled_loss(compiled_model())

BackendCompilerFailed: debug_wrapper raised RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.DoubleTensor [4096]], which is output 0 of DivBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

Set torch._dynamo.config.verbose=True for more information


You can suppress this exception and fall back to eager by setting:
    torch._dynamo.config.suppress_errors = True


In [11]:
# optimizer = rms_torch.Adam(compiled_model.parameters(), lr=1e-3, amsgrad=True)
optimizer = rms_torch.LION(compiled_model.parameters(), lr=1*1e-2)
num_epochs = 10
for epoch in range(num_epochs):
    # Zero out the gradients
    optimizer.zero_grad()

    # Forward pass
    output = compiled_model()
    loss = compiled_loss(output)
    loss.backward()
    for p in model.parameters():
        grad = p.grad  # Get the gradient from the compiled model
        if grad is not None:
            grad.data[:] = rms_torch.riemannian_grad_torch(p.data, grad)
        else:
            raise RuntimeError("No gradient for parameter")
    optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 23.314299400009702
Epoch: 1, Loss: 23.038843048493845
Epoch: 2, Loss: 22.755859174870388
Epoch: 3, Loss: 22.463636181197213
Epoch: 4, Loss: 22.169798842247978


KeyboardInterrupt: 

In [13]:
next(model.parameters())

Parameter containing:
tensor([[-2.5696e-01,  1.7097e-01,  1.9867e-01, -1.2842e-01, -1.0919e-01,
          6.8600e-01, -2.0272e-01, -5.7023e-01],
        [-3.8232e-02, -5.4888e-01,  1.9174e-01,  4.7085e-01,  2.6820e-01,
          4.8821e-01, -2.3459e-02,  3.5774e-01],
        [-3.5027e-01, -3.8501e-01, -1.0177e-01, -1.7628e-01, -3.0537e-01,
          3.9967e-02,  7.6094e-01, -1.1732e-01],
        [-8.9114e-02, -2.2356e-01, -7.4985e-01, -4.1507e-01,  3.4657e-01,
          1.9739e-01, -2.1377e-01,  5.2455e-02],
        [ 1.7096e-01, -1.5171e-01,  5.6047e-01, -7.0626e-01,  3.2317e-01,
          1.1962e-02,  6.7080e-02,  1.6045e-01],
        [-5.7808e-01, -3.9429e-01,  1.8568e-01,  3.4663e-02,  1.0437e-01,
         -4.8289e-01, -4.0521e-01, -2.5770e-01],
        [ 1.4227e-01, -3.0032e-01, -1.6723e-04, -2.2395e-01, -7.6582e-01,
          8.1621e-02, -4.0599e-01,  2.8538e-01],
        [-6.4671e-01,  4.5012e-01,  5.0319e-02, -9.1091e-02, -3.1170e-02,
          1.0248e-01, -1.0116e-02,  5.9729e

In [8]:
compiled_model.reset_params()
optimizer = rms_torch.LION(compiled_model.parameters(), lr=3*1e-4)
num_epochs = 1
for epoch in range(num_epochs):
    # Zero out the gradients
    optimizer.zero_grad()

    # Forward pass
    output = compiled_model()
    loss = compiled_loss(output)
    loss.backward()
    for p in model.parameters():
        grad = p.grad  # Get the gradient from the compiled model
        if grad is not None:
            grad.data[:] = rms_torch.riemannian_grad_torch(p.data, grad)
        else:
            raise RuntimeError("No gradient for parameter")
    optimizer.step()

    print(f"Epoch: {epoch}, Loss: {loss.item()}")

RuntimeError: No gradient for parameter

In [ ]:

from importlib import reload
import sys
sys.path.append("/home/user/project/python/rmsKit")
import rms

import jax 
import numpy as np
import jax.numpy as jnp

sys.path.append("/home/user/project/python/reduce_nsp")
sys.path.append("/home/user/project/python/exact")
import numpy as np

import os
groundstate = np.load("/home/user/project/python/exact/test/out/KH_2x2/Jx_1_Jy_1_Jz_1_h_0/groundstate.npy")
u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
H0 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/0.npy")
H1 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/1.npy")
H2 = -np.load("/home/user/project/python/rmsKit/array/KH/3site/none/Jx_1_Jy_1_Jz_1_hx_0_hz_0/H/2.npy")
x1 = groundstate.copy()
x0 = groundstate.reshape([8] * 4)
x0 = x0.transpose([0, 2, 1, 3]).reshape(-1)
x0 = jnp.array(x0)
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
SH0 = rms.sum_ham(H0, [[0,2]], 4, 8)
SH1 = rms.sum_ham(H1, [[0,1]], 4, 8)
SH2 = rms.sum_ham(H2, [[0,3]], 4, 8)


In [ ]:
_bonds = [[0,0,2], [1,0,1], [2,0,3], [0,1,3], [1,1,0], [2,1,2], [0,2,0], [1,2,3], 
            [2,2,1], [0,3,1], [1,3,2], [2,3,0]]
bonds = [[], [], []]
for bond in _bonds:
    bonds[bond[0]].append(bond[1:])

_H = rms.sum_ham(rms.stoquastic(H0), bonds[0], 4, 8)
_H += rms.sum_ham(rms.stoquastic(H1), bonds[1], 4, 8)
_H += rms.sum_ham(rms.stoquastic(H2), bonds[2], 4, 8)

u2 = np.kron(u, u)
_HU = rms.sum_ham(rms.stoquastic(u2@H0@u2.T), bonds[0], 4, 8)
_HU += rms.sum_ham(rms.stoquastic(u2@H1@u2.T), bonds[1], 4, 8)
_HU += rms.sum_ham(rms.stoquastic(u2@H2@u2.T), bonds[2], 4, 8)

In [ ]:
U = np.kron(u,u)
U = np.kron(U,U)

SH = SH0 + SH1 + SH2

x = jnp.abs(U @ groundstate)
print(x @ _HU @ x)
x = jnp.abs(groundstate)
print(x @ rms.stoquastic(H) @ x)
# print(groundstate @ rms.stoquastic(H) @ groundstate)

# jnp.linalg.eigvalsh(rms.stoquastic(U @ H @ U.T))

-7.052132213843223
-6.22515992356388


In [ ]:
1.763032714130139 * 4

7.052130856520556

In [ ]:
np.linalg.norm(_H - rms.stoquastic(H))

1.579816545884742e-14

In [ ]:
jnp.linalg.eigvalsh(rms.stoquastic(H))

Array([-8.19503796, -7.8031308 , -7.8031308 , ...,  5.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
jnp.linalg.eigvalsh(_H)

Array([-8.19503796, -7.8031308 , -7.8031308 , ...,  5.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
jnp.linalg.eigvalsh(H)

Array([-5.44487522, -5.3283924 , -5.29823654, ...,  6.        ,
        6.        ,  6.        ], dtype=float64)

In [ ]:
state = rms.loss.init_loss(jnp.array(H1), 8, np.float64, "qes", X=jnp.array(x1))
qesLoss = rms.loss.qes_multi

qesLoss([state], jnp.array(u))

Array(0.52065724, dtype=float64)

## With Bug

In [ ]:
import torch
import numpy as np
from torch import nn
import math

u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
device = torch.device("cuda")

def random_unitary_matrix(size, device):
    random_matrix = np.random.randn(size, size)
    q, _ = np.linalg.qr(random_matrix)
    return torch.tensor(u, dtype=torch.float64, device=device)

class CustomModel(nn.Module):
    def __init__(self, H_size, unitary_size):
        super(CustomModel, self).__init__()

        # Calculate the number of unitary matrices required to match the size of 'H'
        n_us = round(math.log2(H_size) / math.log2(unitary_size))
        self.n_us = n_us

        # Initialize the given number of random unitary matrices
        self.us = nn.ParameterList([nn.Parameter(random_unitary_matrix(unitary_size, device), requires_grad=True) for _ in range(n_us)])

    def forward(self):
        # Calculate U as the Kronecker product of all the unitary matrices in 'us'
        U = self.us[0]
        U = torch.kron(U, self.us[1])
        U = torch.kron(U, self.us[2])
        U = torch.kron(U, self.us[3])
        # for i in range(4):
        #     U = torch.kron(U, self.us[i+1])
        # for u in self.us[1:]:
        #     U = torch.kron(U, u)
        return U

def custom_loss(U, H):
    # Calculate U @ H @ U.T
    A = torch.matmul(U, torch.matmul(H, U.T))

    # Calculate the absolute value of the result
    a = torch.max(torch.diag(A)) * torch.eye(A.shape[0], device=device)
    result_abs = -torch.abs(A - a) + a

    # Calculate the minimum eigenvalue
    E = torch.linalg.eigvalsh(result_abs)
    z = torch.exp(-E * 1).sum()
    return torch.log(z)

p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
H = KH.system([2, 2], "3site", p)

# Create an instance of the CustomModel class
model = CustomModel(H.shape[0], 8).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)
# Calculate the output
